In [2]:
from typing import Dict

from prefect import Flow, task, case
from prefect import Parameter

from lume_services.results import Result
from lume_services.tasks import (
    configure_lume_services,
    prepare_lume_model_variables,
    check_local_execution,
    SaveDBResult,
    LoadDBResult,
    LoadFile,
    SaveFile,
)
from lume_services.files import TextFile
from lume_model.variables import InputVariable, OutputVariable
from prefect.storage import Module

from lume_lcls_cu_inj_nn.model import LCLSCuInjNN
from lume_lcls_cu_inj_nn import INPUT_VARIABLES, CU_INJ_MAPPING_TABLE

2023-07-31 18:37:07.144906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# CONFIGURE LUME-SERVICES
# see https://slaclab.github.io/lume-services/workflows/#configuring-flows-for-use-with-lume-services
configure = configure_lume_services.run()

# CHECK WHETHER THE FLOW IS RUNNING LOCALLY
# If the flow runs using a local backend, the results service will not be available
running_local = check_local_execution.run()
#running_local.set_upstream(configure)

In [4]:
def preprocessing_task(input_variables):
    # scale all values w.r.t. impact factor
    for var_name, var in input_variables.items():

        # if name included in scaling factors
        if (
            CU_INJ_MAPPING_TABLE["impact_name"]
            .str.contains(var_name, regex=False)
            .any()
        ):

            # scaled_val = (
            #     var.value
            #     * CU_INJ_MAPPING_TABLE.loc[
            #         CU_INJ_MAPPING_TABLE["impact_name"] == var_name, "impact_factor"
            #     ].item()
            # )

            var.value = var.value

    return input_variables
def evaluate(formatted_input_vars):

    model = LCLSCuInjNN()

    return model.evaluate(formatted_input_vars)

In [5]:
input = {
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    }

In [6]:
    input_variable_parameter_dict = {
        var_name: Parameter(var_name, default=var.default)
        for var_name, var in INPUT_VARIABLES.items()
    }

    # ORGANIZE INPUT VARIABLE VALUES LUME-MODEL VARIABLES
    formatted_input_vars = prepare_lume_model_variables.run(
        input_variable_parameter_dict, INPUT_VARIABLES
    )

    # Perform scaling of variables
    processed_input_vars = preprocessing_task(formatted_input_vars)

    for key in input.keys():
        processed_input_vars[key].value = input[key]

    # # RUN EVALUATION
    #print(processed_input_vars)
    output_variables = evaluate(processed_input_vars)

    # # SAVE RESULTS TO RESULTS DATABASE, requires LUME-services results backend 
    # with case(running_local, False):
    #     # CREATE LUME-services Result object
    #     formatted_result = format_result(
    #         input_variables=processed_input_vars, output_variables=output_variables
    #     )

    #     # RUN DATABASE_SAVE_TASK
    #     saved_model_rep = save_db_result_task(formatted_result)
    #     saved_model_rep.set_upstream(configure)



2023-07-31 18:37:10.094193: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-31 18:37:10.094417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-31 18:37:10.111619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[array([[ 4.13000000e-01,  7.49977244e+00,  2.50000000e+02,
         1.70000000e-01, -7.40000000e-03, -7.40000000e-03,
        -8.89970000e+00,  7.00000000e+07,  4.61470020e+00]])]
1/1 [==============================] - 1s 721ms/step


2023-07-31 18:37:12.020178: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [7]:
output_variables

{'end_n_particle': ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_n_particle', value=9998.101, precision=None, default=None),
 'end_mean_gamma': ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_mean_gamma', value=147.09068, precision=None, default=None),
 'end_sigma_gamma': ScalarOutputVariable(variable_type='scalar', units='1', parent_variable=None, value_range=None, name='end_sigma_gamma', value=0.54239374, precision=None, default=None),
 'end_mean_x': ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_x', value=-1.0433723e-08, precision=None, default=None),
 'end_mean_y': ScalarOutputVariable(variable_type='scalar', units='m', parent_variable=None, value_range=None, name='end_mean_y', value=-4.6587957e-08, precision=None, default=None),
 'end_norm_emit_x': ScalarOutputVariable(variable_type='scalar', units='m'

In [1]:
import torch
import matplotlib.pyplot as plt

from lume_model.utils import variables_from_yaml
from lume_model.torch import LUMEModule, PyTorchModel

In [47]:
# load transformers
input_transformer = torch.load("torch_model/input_transformer.pt")
output_transformer = torch.load("torch_model/output_transformer.pt")

In [48]:
input_variables, output_variables = variables_from_yaml(open("torch_model/variables.yml"))

In [55]:
print('PyTorch Model - ', input_variables.keys())
print('Keras Model - ', input.keys())

input_variables_10 = {}
for key in input.keys():
    input_variables_10[key] = input_variables[key]
input_variables_10

for key in input_variables_10.keys():
    if key not in input_variables:
        print(key)


PyTorch Model -  dict_keys(['distgen:r_dist:sigma_xy:value', 'distgen:t_dist:length:value', 'distgen:total_charge:value', 'SOL1:solenoid_field_scale', 'CQ01:b1_gradient', 'SQ01:b1_gradient', 'L0A_scale:voltage', 'L0A_phase:dtheta0_deg', 'L0B_scale:voltage', 'L0B_phase:dtheta0_deg', 'QA01:b1_gradient', 'QA02:b1_gradient', 'QE01:b1_gradient', 'QE02:b1_gradient', 'QE03:b1_gradient', 'QE04:b1_gradient'])
Keras Model -  dict_keys(['distgen:r_dist:sigma_xy:value', 'distgen:total_charge:value', 'distgen:t_dist:length:value', 'SOL1:solenoid_field_scale', 'CQ01:b1_gradient', 'SQ01:b1_gradient', 'L0A_phase:dtheta0_deg', 'L0A_scale:voltage', 'end_mean_z'])


KeyError: 'end_mean_z'

In [39]:
lume_model = PyTorchModel(
    model_file="torch_model/model.pt",
    input_variables=input_variables_10,
    output_variables=output_variables,
    input_transformers=[input_transformer],
    output_transformers=[output_transformer],
)

In [40]:
lume_module = LUMEModule(
    model=lume_model,
    feature_order=lume_model.features,
    output_order=lume_model.outputs,
)
lume_module

LUMEModule(
  (base_model): Sequential(
    (0): Linear(in_features=16, out_features=100, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=100, out_features=200, bias=True)
    (3): ELU(alpha=1.0)
    (4): Dropout(p=0.05, inplace=False)
    (5): Linear(in_features=200, out_features=200, bias=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.05, inplace=False)
    (8): Linear(in_features=200, out_features=300, bias=True)
    (9): ELU(alpha=1.0)
    (10): Dropout(p=0.05, inplace=False)
    (11): Linear(in_features=300, out_features=300, bias=True)
    (12): ELU(alpha=1.0)
    (13): Dropout(p=0.05, inplace=False)
    (14): Linear(in_features=300, out_features=200, bias=True)
    (15): ELU(alpha=1.0)
    (16): Dropout(p=0.05, inplace=False)
    (17): Linear(in_features=200, out_features=100, bias=True)
    (18): ELU(alpha=1.0)
    (19): Dropout(p=0.05, inplace=False)
    (20): Linear(in_features=100, out_features=100, bias=True)
    (21): ELU(alpha=1.0)
    (22): Linear(in_f

In [41]:
inputs_small = torch.load("info/inputs_small.pt")
outputs_small = torch.load("info/outputs_small.pt")
with torch.no_grad():
    predictions = lume_module(inputs_small)

In [42]:
predictions

tensor([[1.7824e-04, 1.4451e-03, 5.4184e-04, 1.4162e-06, 1.1945e-06],
        [1.7793e-04, 8.5264e-04, 6.5828e-04, 1.1827e-06, 1.1378e-06],
        [2.9690e-04, 2.0847e-04, 7.0719e-04, 1.6769e-06, 9.0121e-07],
        ...,
        [1.3290e-03, 1.5062e-03, 4.2577e-04, 5.8380e-07, 5.8444e-07],
        [1.5413e-03, 1.7313e-03, 4.2518e-04, 5.9533e-07, 5.8237e-07],
        [6.2685e-04, 6.9299e-05, 5.1763e-04, 7.5973e-07, 9.7208e-07]],
       dtype=torch.float64)

In [43]:
import numpy as np
all_values = []
for key in processed_input_vars:
    all_values.append(processed_input_vars[key].value)
all_values = torch.Tensor([np.array(all_values)])
print(all_values)

tensor([[ 4.1300e-01,  7.4998e+00,  2.5000e+02,  1.7000e-01, -7.4000e-03,
         -7.4000e-03, -8.8997e+00,  7.0000e+07,  4.6147e+00]])


In [44]:
input_variables.keys()

dict_keys(['distgen:r_dist:sigma_xy:value', 'distgen:t_dist:length:value', 'distgen:total_charge:value', 'SOL1:solenoid_field_scale', 'CQ01:b1_gradient', 'SQ01:b1_gradient', 'L0A_scale:voltage', 'L0A_phase:dtheta0_deg', 'L0B_scale:voltage', 'L0B_phase:dtheta0_deg', 'QA01:b1_gradient', 'QA02:b1_gradient', 'QE01:b1_gradient', 'QE02:b1_gradient', 'QE03:b1_gradient', 'QE04:b1_gradient'])

In [45]:
output_variables_torch = lume_module(all_values)
output_variables_torch

IndexError: index 9 is out of bounds for dimension 1 with size 9